In [3]:
import os
import sys
os.chdir('/home/zongchen/mmd_flow_cubature/')
sys.path.append('/home/zongchen/mmd_flow_cubature/')
import numpy as np
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
jax.config.update("jax_platform_name", "cpu")

from mmd_flow.distributions import Distribution
from mmd_flow.kernels import gaussian_kernel
from mmd_flow.mmd import mmd_fixed_target
from mmd_flow.gradient_flow import gradient_flow

In [4]:
import numpy as np
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import os
os.chdir("/home/zongchen/mmd_flow_cubature")

import sys
sys.path.append("/home/zongchen/mmd_flow_cubature")
from mmd_flow.distributions import Distribution
from mmd_flow.kernels import gaussian_kernel
from mmd_flow.mmd import mmd_fixed_target
from mmd_flow.gradient_flow import gradient_flow
import mmd_flow.utils
jax.config.update("jax_enable_x64", True)
jax.config.update("jax_platform_name", "cpu")

from tqdm import tqdm

<Figure size 640x480 with 0 Axes>

In [5]:
bandwidth = 1.0
kernel = gaussian_kernel(bandwidth)
covariances = jnp.load('data/mog_covs.npy')
means = jnp.load('data/mog_means.npy')
# means = jnp.zeros((20, 2))
# covariances = jnp.array([jnp.eye(2) for _ in range(20)])
k = 20
weights = jnp.ones(k) / k
distribution = Distribution(kernel=kernel, means=means, covariances=covariances, integrand_name='neg_exp', weights=weights)
mmd_func = mmd_fixed_target(None, kernel, distribution)


In [6]:
for particle_num in [10, 100, 1000, 10000]:
    for seed in tqdm(range(10)):
        temp_list = []
        rng_key = jax.random.PRNGKey(seed)
        iid_samples = distribution.sample(particle_num, rng_key)
        # print(iid_samples)
        mmd = mmd_func(iid_samples)
        temp_list.append(mmd)
    print(np.mean(temp_list), np.std(temp_list))


100%|██████████| 10/10 [00:03<00:00,  2.96it/s]


0.2953346881985013 0.0


100%|██████████| 10/10 [00:03<00:00,  3.14it/s]


0.09471379833141584 0.0


100%|██████████| 10/10 [00:03<00:00,  3.02it/s]


0.023494052125516755 0.0


100%|██████████| 10/10 [00:13<00:00,  1.35s/it]

0.007240658509447128 0.0


In [7]:
def kme_double_RBF_diff_Gaussian(mu_1, mu_2, Sigma_1, Sigma_2, l):
    """
    Computes the double integral a gaussian kernel with lengthscale l, with two different Gaussians.
    
    Args:
        mu_1, mu_2: (D,) 
        Sigma_1, Sigma_2: (D, D)
        l : scalar

    Returns:
        A scalar: the value of the integral.
    """
    D = mu_1.shape[0]
    l_ = l ** 2
    Lambda = jnp.eye(D) * l_
    sum_ = Sigma_1 + Sigma_2 + Lambda
    part_1 = jnp.sqrt(jnp.linalg.det(Lambda) / jnp.linalg.det(sum_))
    sum_inv = jnp.linalg.inv(sum_)
    # Compute exponent: - (1/2) * mu^T * (Σ1 + Σ2 + Lambda)⁻¹ * Γ⁻¹ * mu
    exp_term = -0.5 * ((mu_1 - mu_2).T @ sum_inv @ (mu_1 - mu_2))
    exp_value = jnp.exp(exp_term)
    print(part_1, exp_value)
    result = part_1 * exp_value
    return result


D = 3  # Dimension
mu_1 = jnp.array([1.0, -0.5, 0.3])
mu_2 = jnp.array([0.5, 0.2, -0.1])
Sigma_1 = jnp.array([[1.0, 0.2, 0.1], [0.2, 1.5, 0.3], [0.1, 0.3, 2.0]])
Sigma_2 = jnp.array([[1.2, 0.1, 0.0], [0.1, 1.3, 0.2], [0.0, 0.2, 1.1]])

l = 0.1 # Kernel bandwidth
sample_size = 1000  # Monte Carlo sample sizes
# Compute closed-form solution
closed_form_value = kme_double_RBF_diff_Gaussian(mu_1, mu_2, Sigma_1, Sigma_2, l)
print(closed_form_value)
rng_key = jax.random.PRNGKey(0)
# Generate samples from N(mu, Sigma)
L_1 = jnp.linalg.cholesky(Sigma_1)  # Cholesky decomposition
L_2 = jnp.linalg.cholesky(Sigma_2)  # Cholesky decomposition
rng_key, subkey = jax.random.split(rng_key)
z_1 = jax.random.normal(rng_key, shape=(sample_size, D))  # Standard normal samples
rng_key, subkey = jax.random.split(rng_key)
z_2 = jax.random.normal(rng_key, shape=(sample_size, D))  # Standard normal samples
samples_1 = mu_1 + z_1 @ L_1.T  # Transform to N(mu, Sigma)
samples_2 = mu_2 + z_2 @ L_2.T  # Transform to N(mu, Sigma)
kernel = gaussian_kernel(l)
K = kernel.make_distance_matrix(samples_1, samples_2)
print(K.mean())  # Monte Carlo mean


0.00023260273333841343 0.8134555777527258
0.0001892119908346623
0.00019394465995004295
